In [1]:
import qsharp
import json
from diskcache import Cache
import re_utils

re_utils.DEBUG = True

cache = Cache("~/quant-arith-cache/re-mod-exp")
qsharp.init(project_root="../lib/")

@cache.memoize()
def estimate_resources_mod_exp(op, n):
    est = qsharp.estimate(f"EstimateUtils.RunModExp({n},{op})")
    return json.dumps(est)    

ops_and_max_n = [
  ("QuantumArithmetic.LYY2021.ModExp", "LYY", 2**9),
  ("QuantumArithmetic.LYY2021.ModExpWindowed(_,_,_,_,1)", "LYY-W-1", 128),
  ("QuantumArithmetic.LYY2021.ModExpWindowed(_,_,_,_,11)","LYY-W-11", 2**9),
  ("QuantumArithmetic.LYY2021.ModExpWindowedOptimal", "LYY-W-Opt", 2**9),
  ("QuantumArithmetic.LYY2021.ModExpWindowedMontgomery(_,_,_,_,1)", "LYY-MW-1", 128),
  ("QuantumArithmetic.LYY2021.ModExpWindowedMontgomery(_,_,_,_,11)", "LYY-MW-11", 2**9),
  ("QuantumArithmetic.PG2012.EXP_MOD", "QFT", 128),  
  ("QuantumArithmetic.CG20192.ModExpWindow(_,_,_,_,2,2)", "CG-2019", 2**9)  
]

re_utils.run_re_experiments(
    ops_and_max_n, 
    estimate_resources_mod_exp,
    title='(f) Modular exponentiation')

Charts saved to img/f_modular_exponentiation.png


In [2]:
re_utils.show_re_table(
    ops_and_max_n, 
    estimate_resources_mod_exp, 128)

n=128


,Algorithm,Logical qubits,Physical qubits,Logical depth,Runtime (seconds)
0,LYY,1361,2517458,145729566,1573.879313
1,LYY-W-1,67086,113371772,244054125,2831.027850
2,LYY-W-11,7347,9750190,22193781,221.937810
3,LYY-W-Opt,6310,8453940,19363945,193.639450
4,LYY-MW-1,67351,129915102,292587510,3628.085124
5,LYY-MW-11,7614,11601012,26571302,286.970062
6,QFT,2665,18858410,786327562,9121.399719
7,CG-2019,1359,2265190,25935056,259.350560


In [3]:
re_utils.trendline_analysis(
    ops_and_max_n, 
    estimate_resources_mod_exp, min_n=16)

,Algorithm,Logical qubits,Physical qubits,Logical depth,Runtime (seconds)
0,LYY,1.3452e+01 * n^0.9538,3.3547e+04 * n^0.9067,3.4596e+01 * n^3.1330,1.8063e-04 * n^3.2739
1,LYY-W-1,5.7264e+00 * n^1.9269,3.4088e+03 * n^2.1468,9.4644e+01 * n^3.0451,4.9759e-04 * n^3.2153
2,LYY-W-11,2.0971e+00 * n^1.7011,1.9107e+03 * n^1.8071,1.4133e+01 * n^2.9461,6.6399e-05 * n^3.1072
3,LYY-W-Opt,3.9487e+00 * n^1.5304,3.6171e+03 * n^1.6317,2.3144e+01 * n^2.8054,1.0664e-04 * n^2.9681
4,LYY-MW-1,6.1046e+00 * n^1.9138,3.4278e+03 * n^2.1539,1.3265e+02 * n^3.0110,6.8593e-04 * n^3.1885
5,LYY-MW-11,2.6080e+00 * n^1.6654,2.2314e+03 * n^1.7881,1.7453e+01 * n^2.9399,8.4112e-05 * n^3.0988
6,QFT,2.4629e+01 * n^0.9642,6.0034e+05 * n^0.7497,7.1299e+03 * n^2.4111,4.7093e-02 * n^2.5270
7,CG-2019,1.3236e+01 * n^0.9567,2.5264e+04 * n^0.9361,1.1592e+01 * n^3.0134,5.5008e-05 * n^3.1621
